In [1]:
import json
import pandas as pd
import numpy as np
from telethon.sync import TelegramClient
from telethon.tl.types import PeerUser, PeerChat, PeerChannel, InputMessagesFilterPhotoVideo
from telethon.tl.functions.channels import GetFullChannelRequest
from telethon.errors import ChannelPrivateError
import tqdm
import glob
import pickle

/var/folders/r3/t2r07q612snc9xmhh7_6wkqm0000gn/T/ipykernel_19379/410453825.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('../credentials/credentials.pickle','rb') as file:
    api_id, api_hash, phone, username = pickle.load(file)

In [3]:
edgelist = pd.DataFrame(columns=['forward_from','forward_to','count'])

for path in glob.glob('../*_edgelist/*edgelist*.csv'):
    df = pd.read_csv(path)
    edgelist = pd.concat([edgelist,df])

In [4]:
with open('../dictionaires/id_to_desc.json') as file_desc,\
    open('../dictionaires/id_to_names.json') as file_names:
    id_to_desc = json.load(file_desc)
    id_to_names = json.load(file_names)

In [5]:
id_to_names_int = dict(map(lambda x: (int(x[0]),x[1]), id_to_names.items()))

In [6]:
list_nodes = edgelist['forward_from'].unique()

In [7]:
names_unknown = list(filter(lambda x: x not in id_to_names_int.keys(),list_nodes))

In [8]:
async def get_channel_names(channel_id, names_dict, desc_dict):
    """
    Update names_dict and desc_dict with an element like:

    channel_id : channel name
    channel_id : channel description

    args:

    channel_id: id of channel to be scraped (has to be an integer!)
    names_dict: dict of id-to-names
    desc_dict: dict of id-to-bio
    
    return None
    """
    
    async with TelegramClient(username, api_id, api_hash) as client:
                entity = await client.get_entity(PeerChannel(channel_id))
                full_entity = await client(GetFullChannelRequest(channel=entity))

                names_dict[channel_id] = entity.title #get group name
                desc_dict[channel_id] = full_entity.full_chat.about #get group bio

    
    return None

In [9]:
start_value = 0
final_value = len(names_unknown)

In [ ]:
for index in tqdm.tqdm(range(start_value, final_value)):
    codice = names_unknown[index]
    try:
        await get_channel_names(int(codice), id_to_names, id_to_desc)
    except(ChannelPrivateError, ValueError) as e:
        print(e)

In [13]:
with open('../dictionaires/id_to_desc.json','w') as file_desc,\
    open('../dictionaires/id_to_names.json','w') as file_names:
    json.dump(id_to_desc, file_desc)
    json.dump(id_to_names, file_names)